In [1]:
%matplotlib inline

from pathlib import Path

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import nltk
from nltk import word_tokenize          
from nltk.stem.snowball import EnglishStemmer 
import matplotlib.pylab as plt
from dmba import printTermDocumentMatrix, classificationSummary, liftChart

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/JAE111/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 20.1

## a

1. !
2. <br /> 
3. <font size="3> 
4. &qout; 
5. </font>
6. .
7. ?
8. ,
9. \<img title="smile" alt="smile" src="\url.........}" \>

## b

Thanks, John, will, be, for, to, through, on, their, and, the ...

## c

Illustrations, demos, students, project, where, find

## d

html 문서의 태그로 사용되는 단어들이 용어로 인식되면 행렬이 비대해지고, 잡음이 생길 수 있다.

# 20.2

## a

In [3]:
# Step 1: import and label records
corpus = []
label = []
with ZipFile('AutoAndElectronics.zip') as rawData:
    for info in rawData.infolist():
        if info.is_dir(): 
            continue
        label.append(1 if 'rec.autos' in info.filename else 0)
        corpus.append(rawData.read(info))


## b

In [6]:
# Step 2: preprocessing (tokenization, stemming, and stopwords)
class LemmaTokenizer(object):
    def __init__(self):
        self.stemmer = EnglishStemmer()
        self.stopWords = set(ENGLISH_STOP_WORDS)
    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in word_tokenize(doc) 
                if t.isalpha() and t not in self.stopWords]

preprocessor = CountVectorizer(tokenizer=LemmaTokenizer(), encoding='latin1')
preprocessedText = preprocessor.fit_transform(corpus)

불필요하고 잡음이 많은 단어들이 다수 포함되면서 행렬이 비대해지고 예측성능이 떨어지고 모델형성에 시간이 많이 들 것이다.

## c

In [7]:
# Step 3: TF-IDF and latent semantic analysis
tfidfTransformer = TfidfTransformer()
tfidf = tfidfTransformer.fit_transform(preprocessedText)

<2000x13515 sparse matrix of type '<class 'numpy.float64'>'
	with 159748 stored elements in Compressed Sparse Row format>

In [10]:


# Extract 20 concepts using LSA ()
svd = TruncatedSVD(20)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

lsa_tfidf = lsa.fit_transform(tfidf)

텍스트의 관리용이성을 크게 증진시키고 잡음을 줄여서 예측성능을 높일 것이다.

In [16]:
# split dataset into 60% training and 40% test set
Xtrain, Xtest, ytrain, ytest = train_test_split(lsa_tfidf, label, test_size=0.4, random_state=42)

# run logistic regression model on training
logit_reg = LogisticRegression(solver='lbfgs')
logit_reg.fit(Xtrain, ytrain)

# print confusion matrix and accuracty
classificationSummary(ytest, logit_reg.predict(Xtest))

Confusion Matrix (Accuracy 0.9575)

       Prediction
Actual   0   1
     0 389   8
     1  26 377


## d

In [18]:
from sklearn.ensemble import RandomForestClassifier
# split dataset into 60% training and 40% test set
Xtrain, Xtest, ytrain, ytest = train_test_split(lsa_tfidf, label, test_size=0.4, random_state=42)

# run randomforest classifier model on training
forest = RandomForestClassifier()
forest.fit(Xtrain, ytrain)

# print confusion matrix and accuracy
classificationSummary(ytest, forest.predict(Xtest))

Confusion Matrix (Accuracy 0.9625)

       Prediction
Actual   0   1
     0 390   7
     1  23 380


accuracy가 0.9587이었던 rogistic 회귀보다 높은 성능을 보임

# 20.3

## a

In [20]:
raw = pd.read_csv("./farm-ads.csv")
raw

,1 ad-jerry ad-bruckheimer ad-chase ad-premier ad-sept ad-th ad-clip ad-bruckheimer ad-chase page found
0,-1 ad-rheumatoid ad-arthritis ad-expert ad-tip...
1,-1 ad-rheumatologist ad-anju ad-varghese ad-yo...
2,-1 ad-siemen ad-water ad-remediation ad-water ...
3,-1 ad-symptom ad-muscle ad-weakness ad-genetic...
4,1 ad-animal ad-animal ad-wild ad-sa ad-officia...
...,...
4137,-1 ad-affordable ad-ivf ad-cost ad-efficient a...
4138,1 ad-mozypro ad-business ad-backup ad-affordab...
4139,1 ad-oster ad-line ad-clipper ad-oster ad-fact...
4140,-1 ad-synrevoice ad-schoolconnect ad-trust ad-...


1 ad-animal ad-animal ad-wild ad-sa ad-officia...  
1 ad-vet ad-online ad-veterinarian ad-online a...  
-1 ad-rheumatoid ad-arthritis ad-expert ad-tip...  
-1 ad-rheumatologist ad-anju ad-varghese ad-yo...  

In [21]:
class LemmaTokenizer(object):
    def __init__(self):
        self.stemmer = EnglishStemmer()
        self.stopWords = set(ENGLISH_STOP_WORDS)
    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in word_tokenize(doc) 
                if t.isalpha() and t not in self.stopWords]

preprocessor = CountVectorizer(tokenizer=LemmaTokenizer(), encoding='latin1')
preprocessedText = preprocessor.fit_transform(corpus)

In [22]:
tfidfTransformer = TfidfTransformer()
tfidf = tfidfTransformer.fit_transform(preprocessedText)

In [23]:
svd = TruncatedSVD(20)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

lsa_tfidf = lsa.fit_transform(tfidf)

In [25]:
## tfidf

printTermDocumentMatrix(preprocessor, tfidf)

          S1   S2   S3   S4   S5   S6   S7   S8   S9  S10  ...  S1991  S1992  \
aa       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
aaa      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
aaaaa    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
aaahh    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
aaltern  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
zurich   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
zwar     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
zx       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
zyxel    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
þ        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   

         S1993  S1994  S1995  S1996  S1

### i

희소행렬이다.

### ii

In [33]:
printTermDocumentMatrix(preprocessor, tfidf[45])

               S1
aa       0.000000
aaa      0.000000
aaaaa    0.000000
aaahh    0.000000
aaltern  0.000000
...           ...
zurich   0.000000
zwar     0.000000
zx       0.000000
zyxel    0.000000
þ        0.157242

[13515 rows x 1 columns]


p 문자가 S1문장에서 나온 빈도수와 모든 문장들에서 p가 들어간 문장의 비율의 곱이 0.157...이다.

## b

주성분분석에서 여러 변수들에서 최대한 정보 손실이 안 되도록 차원을 축소한 것 처럼, 여러 용어들을 최대한 그 용어들의 정보를 다 포함한 상태로 더 적은 개념들로 함축시킬 수 있다. 

## c

In [46]:
# split dataset into 60% training and 40% test set
Xtrain, Xtest, ytrain, ytest = train_test_split(lsa_tfidf, label, test_size=0.4, random_state=42)

# run logistic regression model on training
logit_reg = LogisticRegression(solver='lbfgs')
logit_reg.fit(Xtrain, ytrain)

# print confusion matrix and accuracty
classificationSummary(ytest, logit_reg.predict(Xtest))

Confusion Matrix (Accuracy 0.9637)

       Prediction
Actual   0   1
     0 390   7
     1  22 381


유효성??

## d

텍스트의 관리용이성을 크게 증진시키고 잡음을 줄여서 예측성능을 높이기 위해서